In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2
import utils
import webdataset as wds

subj = 1

data_path = "/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset"


train_url = f"{data_path}/webdataset_avg_split/train/train_subj0{subj}_{{0..17}}.tar,{data_path}/webdataset_avg_split/val/val_subj0{subj}_0.tar"
val_url = f"{data_path}/webdataset_avg_split/test/test_subj0{subj}_{{0..1}}.tar"
print(train_url,"\n",val_url)
meta_url = f"{data_path}/webdataset_avg_split/metadata_subj0{subj}.csv"
num_train = 8559 + 300
num_val = 982

cache_dir="/tmp/wds-cache"
my_split_by_node = (lambda urls: urls)
train_data = wds.WebDataset(train_url, resampled=True, cache_dir=cache_dir, nodesplitter=my_split_by_node)

/home/dongho/anaconda3/envs/temp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split/train/train_subj01_{0..17}.tar,/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split/val/val_subj01_0.tar 
 /home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset/webdataset_avg_split/test/test_subj01_{0..1}.tar


In [3]:
# import json 
# import pandas as pd

# metadata = pd.read_csv(meta_url)

In [4]:
# print('Prepping train and validation dataloaders...')
# train_dl, val_dl, num_train, num_val = utils.get_dataloaders(
#     32,
#     'images',
#     num_devices=torch.cuda.device_count(),
#     num_workers=1, 
#     train_url=train_url,
#     val_url=val_url,
#     meta_url=meta_url,
#     num_train=num_train,
#     num_val=num_val,
#     val_batch_size=1,
#     seed=0,
#     voxels_key='nsdgeneral.npy',
#     to_tuple=["voxels", "images", "coco", "brain_3d"],
# )

In [5]:
# '''
# voxels: (B, 3, 15724) # may vary slightly
# image: (B, 3, 256, 256) 
# cocoid: (B, 1)
# brain_3d: (B, 3, 81, 104, 83) # may vary slightly
# '''

# for i, (voxels, image, cocoid, brain_3d) in enumerate(train_dl):
#     if (i==1): break
#     print(f"voxels.shape: {voxels.shape}")
#     print(f"image.shape: {image.shape}")
#     print(f"cocoid.shape: {cocoid.shape}")
#     print(f"brain_3d.shape: {brain_3d.shape}")

#     repeat_index = i % 3
#     for j in range(4):
#         print(voxels[j, repeat_index])
#         plt.figure(figsize=(3, 3))
#         plt.imshow(utils.torch_to_Image(image[j]))
#         plt.show()
#         print(f"cocoid: {cocoid[j]}")
#         print(brain_3d[j][repeat_index])

        


# 모든 subject 대상 dataloader 만들기
- cocoId로 EMOTIC과 겹치는 애들 뽑기
- brain_3d and valence pair 만들기
- train, val, test
    - train, val = 8:2
    - test: 공통 이미지 보여준 애들로

In [6]:
# load package
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import scipy.io

file_name_nsd_stim = './nsd_stim_info_merged.csv'
file_name_emotic_annot = './emotic_annotations.mat'

## get EMOTIC data
data = scipy.io.loadmat(file_name_emotic_annot, simplify_cells=True)
emotic_data = data['train'] + data['test'] + data['val']
emotic_coco_data = [x for x in emotic_data if x['original_database']['name']=='mscoco']
coco_id = [x['original_database']['info']['image_id'] for x in emotic_coco_data]
annotations = [x['person'] for x in emotic_coco_data] 
emotic_annotations = []
for annot in annotations:
    annot = [annot] if type(annot)==dict else annot

    valence = []; arousal = []; dominance = []
    for person in annot:
        person = person['annotations_continuous']
        person = [person] if type(person)==dict else person
        valence += [np.mean([x['valence'] for x in person])]
        arousal += [np.mean([x['arousal'] for x in person])]
        dominance += [np.mean([x['dominance'] for x in person])]
    emotic_annotations += [{ 'valence':valence, 'arousal':arousal, 'dominance':dominance}]

emotic_annotations = dict(zip(coco_id, emotic_annotations))

# define function
get_emotic_annot       = lambda coco_id, metric_type: np.mean(emotic_annotations[coco_id][metric_type])
get_emotic_annot_indiv = lambda coco_id, metric_type: emotic_annotations[coco_id][metric_type]

## get NSD data
df = data = pd.read_csv(file_name_nsd_stim)
nsd_id = df['Unnamed: 0'].values
nsd_cocoid = df['cocoId'].values
nsd_cocosplit = df['cocoSplit'].values
nsd_isshared = df['shared1000'].values


## target NSD data
joint_cocoid = nsd_cocoid[np.isin(nsd_cocoid, list(emotic_annotations.keys()))]
# print(target_cocoid)
print("The Num of intersection between NSD and EMOTIC dataSet:", len(joint_cocoid))

"""
1. TrainSet: NSD 데이터셋에서 개별적으로 보여준 것
2. TestSet: NSD 데이터셋에서 공통으로 보여준 것
3. 공통으로, 사람 한명 있는 데이터 사용
 - `len(person)==1`
 """
# 사람 한명 있는 데이터
target_cocoid = [coco_id for coco_id in joint_cocoid if len(get_emotic_annot_indiv(coco_id, 'valence')) == 1]
print("The number of joint pictures which reveals only one person:", len(target_cocoid))
train_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  ~nsd_isshared]
print(f"cocoid for train, #: {len(train_cocoid)}")
test_cocoid = nsd_cocoid[np.isin(nsd_cocoid, target_cocoid) &  nsd_isshared]
print(f"cocoid for test, #: {len(test_cocoid)}")

# data for common image shown to all subjects
# regarding as test set
test_df = df[df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]
train_df =  df[~df[['subject1', 'subject2', 'subject3', 'subject4', 'subject5', 'subject6', 'subject7', 'subject8']].all(axis=1)]

for i in range(1, 9):
    current_subject = f"subject{i}"
    curr_df = train_df[train_df[current_subject] == True] # filter only shown to curr_subject

    # 현재 subject의 데이터에서 train_cocoid와 겹치는 데이터 찾기
    # 즉 현재 subject에 보여준 사진 중 cocoid가 EMOTIC dataset에 사용되었던 것 필터링
    target_train_df = curr_df[np.isin(curr_df["cocoId"], train_cocoid)]
    print(f"data of {current_subject}: {target_train_df.shape}")

target_test_df = test_df[np.isin(test_df["cocoId"], test_cocoid)]

The Num of intersection between NSD and EMOTIC dataSet: 9413
The number of joint pictures which reveals only one person: 7569
cocoid for train, #: 7457
cocoid for test, #: 112
data of subject1: (960, 41)
data of subject2: (900, 41)
data of subject3: (910, 41)
data of subject4: (968, 41)
data of subject5: (967, 41)
data of subject6: (932, 41)
data of subject7: (903, 41)
data of subject8: (917, 41)


In [7]:
def reshape_brain3d(brain_3d, target_shape=(96, 96, 96)):
    # Initialize pad_width
    pad_width = [(0, 0)] * 3  # For 3D brain_3day

    # Calculate padding needed for each dimension
    for i in range(3):
        current_size = brain_3d.shape[i]
        if current_size < target_shape[i]:
            # Calculate padding
            total_pad = target_shape[i] - current_size
            pad_before = total_pad // 2
            pad_after = total_pad - pad_before
            pad_width[i] = (pad_before, pad_after)

    # Apply padding
    brain_3d = np.pad(brain_3d, pad_width=pad_width, mode='constant', constant_values=0)

    # Apply truncation if necessary
    brain_3d = brain_3d[:target_shape[0], :target_shape[1], :target_shape[2]]

    # print("reshaped brain_3d shape", brain_3d.shape)
    return brain_3d

In [9]:
import braceexpand
import math
import random
import webdataset as wds
import pprint as pp

def get_dataloaders(
    batch_size,
    target_cocoid,
    image_var='images',
    num_devices=None,
    train_data_urls=None,
    val_data_urls=None,
    test_data_urls=None,
    num_data=None,
    seed=0,
    voxels_key="nsdgeneral.npy",
    to_tuple=["voxels", "images", "coco", "brain_3d"],
):
    """
    train_data_urls: array of train data url
    val_data_urls: array of val data url
    test_data_urls: array of test data url
    target_cocoid(coco id from both NSD & EMOTIC dataset)
    
    out: three dataloaders that all returns (brain3d, valence)
    """

    print("Getting dataloaders...")
    assert image_var == 'images'
    
    def my_split_by_node(urls):
        return urls
    
    # data_url = list(braceexpand.braceexpand(data_url))

    if num_devices is None:
        num_devices = torch.cuda.device_count()
    
        
    print(f"in utils.py: num_devices: {num_devices}")
    global_batch_size = batch_size * num_devices
    num_batches = math.floor(num_data / global_batch_size)
    # num_worker_batches = math.floor(num_batches / num_workers)
    # if num_worker_batches == 0: num_worker_batches = 1

    print("\nnum_data",num_data)
    print("global_batch_size",global_batch_size)
    print("batch_size",batch_size)
    print("num_batches",num_batches)

    def filter_by_cocoId(sample):
        # sample: ("voxels", "images", "cocoid", "brain_3d")
        _, _, cocoid, _ = sample
        cocoid = cocoid[-1]
        return (cocoid in target_cocoid)

    def map_brain_valence_pair(sample):
        # sample: ("voxels", "images", "cocoid", "brain_3d")
        # add corresponding valence
        # make sure all brain_3d shape is same
        # out: brain_3d, valence

        _, _, cocoid, brain_3d = sample
        cocoid = cocoid[-1]
        valence = get_emotic_annot(cocoid, 'valence')
        brain_3d = np.mean(brain_3d, axis=0) # (*, *, *)
        brain_3d = reshape_brain3d(brain_3d, target_shape=(96, 96, 96)) # (96, 96, 96)
        
        return brain_3d, valence
    
    ### train ###
    
    train_target_urls = []
    for url in train_data_urls:
        train_target_urls += list(braceexpand.braceexpand(url))
    print("len(train_target_urls):", len(train_target_urls))

    # for url in data_urls:
    train_dataset = wds.WebDataset(train_target_urls, resampled=True, nodesplitter=my_split_by_node)\
        .shuffle(500, initial=500, rng=random.Random(seed))\
        .decode("torch")\
        .rename(images="jpg;png", voxels=voxels_key, trial="trial.npy", coco="coco73k.npy", reps="num_uniques.npy", brain_3d = "wholebrain_3d.npy")\
        .to_tuple(*to_tuple)\
        .select(filter_by_cocoId)\
        .map(map_brain_valence_pair)\
        .batched(batch_size, partial=True)\
        # .with_epoch(num_worker_batches)

    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=None, num_workers=1, shuffle=False)       

    ### val ###
    
    val_target_urls = []
    for url in val_data_urls:
        val_target_urls += list(braceexpand.braceexpand(url))
    print("len(val_target_urls):", len(val_target_urls))

    # for url in data_urls:
    val_dataset = wds.WebDataset(val_target_urls, resampled=True, nodesplitter=my_split_by_node)\
        .shuffle(500, initial=500, rng=random.Random(seed))\
        .decode("torch")\
        .rename(images="jpg;png", voxels=voxels_key, trial="trial.npy", coco="coco73k.npy", reps="num_uniques.npy", brain_3d = "wholebrain_3d.npy")\
        .to_tuple(*to_tuple)\
        .select(filter_by_cocoId)\
        .map(map_brain_valence_pair)\
        .batched(batch_size, partial=True)\
        # .with_epoch(num_worker_batches)

    val_dl = torch.utils.data.DataLoader(val_dataset, batch_size=None, num_workers=1, shuffle=False)       

    ### test ###
    
    test_target_urls = []
    for url in test_data_urls:
        test_target_urls += list(braceexpand.braceexpand(url))
    print("len(test_target_urls):", len(test_target_urls))

    # for url in data_urls:
    test_dataset = wds.WebDataset(test_target_urls, resampled=True, nodesplitter=my_split_by_node)\
        .shuffle(500, initial=500, rng=random.Random(seed))\
        .decode("torch")\
        .rename(images="jpg;png", voxels=voxels_key, trial="trial.npy", coco="coco73k.npy", reps="num_uniques.npy", brain_3d = "wholebrain_3d.npy")\
        .to_tuple(*to_tuple)\
        .select(filter_by_cocoId)\
        .map(map_brain_valence_pair)\
        .batched(batch_size, partial=True)\
        # .with_epoch(num_worker_batches)

    test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=None, num_workers=1, shuffle=False)       
    
    return train_dl, val_dl, test_dl

In [11]:
### load dataloaders

data_path = "/home/juhyeon/fsx/proj-medarc/fmri/natural-scenes-dataset"

train_urls = []
for subj in [1, 2, 5, 7]:
    train_url = f"{data_path}/webdataset_avg_split/train/train_subj0{subj}_" + "{0..17}.tar"
    train_urls.append(train_url)

    
val_urls = []
for subj in [1, 2, 5, 7]:
    val_url = f"{data_path}/webdataset_avg_split/val/val_subj0{subj}_0.tar"
    val_urls.append(val_url)

test_urls = []
for subj in [1, 2, 5, 7]:
    test_url = f"{data_path}/webdataset_avg_split/test/test_subj0{subj}_" + "{0..1}.tar"
    test_urls.append(test_url)
    

train_dl, val_dl, test_dl = get_dataloaders(
    batch_size=32,
    target_cocoid=target_cocoid,
    num_devices=torch.cuda.device_count(),
    train_data_urls=train_urls,
    val_data_urls=val_urls,
    test_data_urls=test_urls,
    num_data=3730,
    seed=0,
    voxels_key='nsdgeneral.npy',
    to_tuple=["voxels", "images", "coco", "brain_3d"],
)

train_dl, val_dl, test_dl

Getting dataloaders...
in utils.py: num_devices: 4

num_data 3730
global_batch_size 128
batch_size 32
num_batches 29
len(train_target_urls): 72
len(val_target_urls): 4
len(test_target_urls): 8


(<torch.utils.data.dataloader.DataLoader at 0x7f2e084484f0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f2e555f67a0>)

In [13]:
print("data in trainloader")
for i, (brain_3d, valence) in enumerate(train_dl):
    if (i == 1): break
    for j in range(5):
        print(brain_3d[j].shape)
        print(valence[j])
print("data in valloader")
for i, (brain_3d, valence) in enumerate(val_dl):
    if (i == 1): break
    for j in range(5):
        print(brain_3d[j].shape)
        print(valence[j])
print("data in testloader")
for i, (brain_3d, valence) in enumerate(test_dl):
    if (i == 1): break
    for j in range(5):
        print(brain_3d[j].shape)
        print(valence[j])

data in trainloader
torch.Size([96, 96, 96])
tensor(7., dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(4., dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(6., dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(2., dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(5.3333, dtype=torch.float64)
data in valloader
torch.Size([96, 96, 96])
tensor(6.3333, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(5.5000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(5.5000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(5.5000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(6.3333, dtype=torch.float64)
data in testloader
torch.Size([96, 96, 96])
tensor(6.6000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(8., dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(6.6000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(5.5000, dtype=torch.float64)
torch.Size([96, 96, 96])
tensor(6., dtype=torch.float64)


In [19]:
# len(train_dl.dataset), len(val_dl.dataset), len(test_dl.dataset),
num_val_data = 0
for i, (brain_3d, valence) in enumerate(val_dl):
    num_val_data += 32
    print(num_val_data)

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2e515a3ac0>
Traceback (most recent call last):
  File "/home/dongho/anaconda3/envs/temp/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/dongho/anaconda3/envs/temp/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/home/dongho/anaconda3/envs/temp/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2e515a3ac0>
Traceback (most recent call last):
  File "/home/dongho/anaconda3/envs/temp/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/dongho/anaconda3/envs/temp/lib/python3.10/s

512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648


KeyboardInterrupt: 

In [35]:
test_url = f"{data_path}/webdataset_avg_split/test/test_subj0{subj}_{{0..1}}.tar"